# Test GCS

Jupyter notebook for testing the Geochemical Classification System.

In [ ]:
#setup system
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as spx
from scipy.stats import linregress
from matplotlib.colors import LinearSegmentedColormap, to_hex

#for hysteresis analysis
import sys
sys.path.append('../hygcs')
import gcs as gcs

In [ ]:
# READ DATA
#Hydro-Meteo data
Qx = pd.read_csv('test_hyd.csv', index_col=0)
Qx.index = pd.to_datetime(Qx.index)
Qx['time'] = Qx.index

#Physicochemical data
pcd = pd.read_csv('test_gc.csv',index_col=0)
pcd.date = pd.to_datetime(pcd.date)

In [ ]:
# PERFORM ANALYSIS
classified = gcs.classify_geochemical_phase(pcd, sites=['C4', 'A12', 'B3', 'B4'], flow_highres=Qx, ccol='PLI', qcol='Q_mLs', use_highres=True)

# phase sequence plot
figts2 = gcs.create_phase_sequence_plot(classified, sites=['C4', 'A12', 'B3', 'B4'])
figts2.update_xaxes(range=[Qx.index[0],Qx.index[-1]])
figts2.show()


In [ ]:
#in case needed to keep track of all columns
classified.columns

In [ ]:
# some example plots
#px.line(classified, color='site_id', x='end_date',y='window_HI_zuecco', template='none')
#px.line(classified, color='site_id', x='end_date',y='window_HI_harp', template='none')
px.scatter(classified, color='site_id', x='end_date',y='geochemical_phase',size='phase_confidence', template='none')

In [ ]:
gcs.create_hysteresis_summary_stats(classified,['B3'],'PLI','harp')

In [ ]:
gcs.create_hysteresis_plot(classified,['B3'],ccol='PLI',qcol='Q_mLs',compound='PLI',conc_unit='')

In [ ]:
# create a multi compound plot
import gcs_visualization as gcsv
fig = gcsv.create_multi_compound_hysteresis_plot(
    pcd, Qx, sites=['A12', 'B4', 'B3'],
    ccols=['Cd_mgL', 'Zn_mgL','Fe_mgL','PLI'],
    compounds=['Cd', 'Zn','Fe','PLI'],
    conc_units=['mg L⁻¹', 'mg L⁻¹','mg L⁻¹',''],
    qcol='Q_mLs', flow_unit='mL s⁻¹',
   cxmin=[0.02, 1.,0.01,20.], cxmax=[4., 600.0, 30., 1600.],
    qxmin=4.,
    hi_method='zuecco'  # Use Lloyd's HI for styling - or zuecco
)
fig.update_layout(template='none')
fig.show()
#fig.write_image('hysteresis_plot_zuecco2.pdf', width=1200, height=1400)

In [ ]:
#further plots
figcvc = gcs.create_diagnostic_plot(classified)
figcvc.show()
#figcvc.write_image('cvc_cvq_PLI.pdf',width=1200,height=400)



In [ ]:
dfx = classified.loc[classified.compound=='PLI']
figcvs = px.line(dfx, color='site_id',x='start_date',y='cq_slope_loglog',template='none')
figcvs.update_traces(line=dict(color='rgba(55, 50, 50, 0.25)', width=1))
#figcvs.update_layout(showlegend=False)

figcvs.add_trace(go.Scatter(x=dfx.loc[dfx.site_id.str.contains('C4'),'start_date'], y=dfx.loc[dfx.site_id.str.contains('C4'),'cq_slope_loglog'], mode='lines+markers', name='Site 1', line_color='#1f77b4'))
figcvs.add_trace(go.Scatter(x=dfx.loc[dfx.site_id.str.contains('A12'),'start_date'], y=dfx.loc[dfx.site_id.str.contains('A12'),'cq_slope_loglog'], mode='lines+markers', name='Site 2', line_color='#ff7f0e'))
figcvs.add_trace(go.Scatter(x=dfx.loc[dfx.site_id.str.contains('B3'),'start_date'], y=dfx.loc[dfx.site_id.str.contains('B3'),'cq_slope_loglog'], mode='lines+markers', name='Site 3B', line_color='#2ca02c'))
figcvs.add_trace(go.Scatter(x=dfx.loc[dfx.site_id.str.contains('B4'),'start_date'], y=dfx.loc[dfx.site_id.str.contains('B4'),'cq_slope_loglog'], mode='lines+markers', name='Site 3A', line_color='#d62728'))

labels_to_show_in_legend = ['Site 1', 'Site 2', 'Site 3A', 'Site 3B']
for trace in figcvs['data']: 
    if (not trace['name'] in labels_to_show_in_legend):
        trace['showlegend'] = False
figcvs.update_layout(xaxis_range=[Qx.index[0],Qx.index[-1]], xaxis_title='', legend_title='Sites')
figcvs.show()

#SIx = 'C4'
#figXX = px.line(dfx.loc[dfx.site_id==SIx],x='start_date',y='highres_window_hyst_zuecco_h_index',template='none')
#figXX.update_traces(line=dict(color='rgba(55, 50, 50, 0.25)', width=1))
#
#figXX.add_trace(go.Scatter(x=Qx.index, y=0.001*Qx['Site 1'], mode='lines', name='Q (Ls-1)', line_color='#0072B2'))
#figXX.add_trace(go.Scatter(x=meteod['scPDSI'].index, y=0.1*meteod['scPDSI'], mode='lines', name='Wetness (x0.1)', line_color='#56B4E9'))
#figXX.add_trace(go.Scatter(x=pcd.loc[pcd.site_id.str.contains(SIx),'date'], y=0.001*pcd.loc[pcd.site_id.str.contains(SIx),'PLI'], mode='lines+markers', name='PLI (x0.001)', line_color="#E40D71"))
#
#figXX.add_trace(go.Scatter(x=dfx.loc[dfx.site_id==SIx,'start_date'], y=dfx.loc[dfx.site_id==SIx,'highres_window_hyst_zuecco_h_index'], mode='lines+markers', name='Hi Zuecco', line_color='#E69F00'))
#figXX.add_trace(go.Scatter(x=dfx.loc[dfx.site_id==SIx,'start_date'], y=dfx.loc[dfx.site_id==SIx,'highres_window_hyst_harp_area'], mode='lines+markers', name='Hi HARP area', line_color="#E66000"))
#figXX.add_trace(go.Scatter(x=dfx.loc[dfx.site_id==SIx,'start_date'], y=0.2*dfx.loc[dfx.site_id==SIx,'CVc_CVq'], mode='lines+markers', name='CVc_CVq (x0.2)', line_color="#7FDE50"))
#figXX.add_trace(go.Scatter(x=dfx.loc[dfx.site_id==SIx,'start_date'], y=0.2*dfx.loc[dfx.site_id==SIx,'cq_slope_loglog'], mode='lines+markers', name='C_Q_slope (x0.2)', line_color='#009E73'))
#
#figXX.update_layout(xaxis_range=[Qx.index[0],Qx.index[-1]], xaxis_title='', yaxis_title='Scaled Y-Axes',yaxis_range=[-0.8,1.4], title='Site 1 PLI')
#figXX.show()
#figXX.write_image('1_all_index.pdf',width=1200,height=400)